# Clone Group Source to Target
In this notebook, a group will be cloned from the source organization to a target organization. 

## Getting Started:
1. Find the group IDs for the template groups. These can be found by navigating to the group page and looking in the URL (for example, if the URL is http://envisioning.maps.arcgis.com/home/group.html?id=a7903db4086641b98570bce5856a6364#overview, the group ID is "a7903db4086641b98570bce5856a6364". Example set-up in the project organization:
    - test-group (ID "a7903db4086641b98570bce5856a6364") contains:
        - Sample Layer
        - Sample Map
    - test-group-2 (ID "4d7ff4f81d6340428ef290b7de801204") contains:
        - Sample WebApp
2. Change the variables in the first code cell. Example variables:
    - SOURCE_URL = "https://envisioning.maps.arcgis.com"
    - SOURCE_USERNAME = "admin"
    - SOURCE_GROUP_IDS = ["a7903db4086641b98570bce5856a6364", "4d7ff4f81d6340428ef290b7de801204"]
    - TARGET_URL = "https://envisioning.maps.arcgis.com"
    - TARGET_USERNAME = "admin"
    - COPY_PREFIX = "ACME"
3. Run the notebook cell by cell. 
    - In code cell 3, you will need to enter the password for the source username when prompted
    - In code cell 4, you will need to enter the password for the target username when prompted
4. After running this notebook, the source group will be cloned into a new target organization group. 

In [ ]:
# Global Variables set by user **change these values before running**

# URL of the portal with the template content
SOURCE_URL = "https://my-enterprise.maps.arcgis.com"

# username for source portal log-on (admin)
SOURCE_USERNAME = "my_user_name"

# template content groups that will be shared with delivery org #aec-test-group, aec-test-group-2
SOURCE_GROUP_ID = "6fd586038d76406594d72ed2cb348e41"

# URL of the portal to receive the template content
TARGET_URL = "https://my-other-enterprise.maps.arcgis.com"

# username for the target portal log-on (admin)
TARGET_USERNAME

In [ ]:
# import libraries
from arcgis.gis import GIS
from IPython.display import display

In [ ]:
print('Logging into {}'.format(SOURCE_URL))
source = GIS(SOURCE_URL, SOURCE_USERNAME)

In [ ]:
print('Logging into {}'.format(TARGET_URL))
target = GIS(TARGET_URL,TARGET_USERNAME)

In [ ]:
# Search for group by ID
source_groups = source.groups.search(SOURCE_GROUP_ID)
source_groups

In [ ]:
# Get group object
source_group = source_groups[0]
source_group

In [ ]:
# Get group items
source_items = source_group.content()
source_items

In [ ]:
import tempfile
if not target.groups.search(source_group.title):
    try:
        with tempfile.TemporaryDirectory() as temp_dir:
            thumbnail_file = source_group.download_thumbnail(temp_dir)
            
            #create a group in the target portal with all the properties of the group in the source
            target_group = target.groups.create(title = source_group.title,
                                                 tags = source_group.tags,
                                                 description = source_group.description,
                                                 snippet = source_group.snippet,
                                                 access = source_group.access, 
                                                 thumbnail= thumbnail_file,
                                                 is_invitation_only = True,
                                                 sort_field = 'avgRating',
                                                 sort_order ='asc',
                                                 is_view_only=True)
            #display the group
            display(target_group)
            
    except Exception as e:
        print('Group {} could not be created'.format(source_group.title))
        print(e)
else:
    print('Group {} already exists in the portal'.format(source_group.title))
    target_group = target.groups.search(source_group.title)[0]

In [ ]:
#making a list for the items to be cloned in the target portal
items_to_be_cloned = list(source_items)

#checking for the presence of the item in the target portal 
for item in source_items:
    searched_items = target.content.search(query='title:'+item.title, item_type = item.type)   
    
    for s_item in searched_items:
        
        if s_item.title == item.title:
            
            #if an item is not a part of the group in the target portal then share it 
            if s_item not in target_group.content():
                s_item.share(groups= [target_group])
            
            #remove the already existing item from the list of items to be cloned
            items_to_be_cloned.remove(item)                
            
            #display the item
            display(s_item)      
                     
            break

In [ ]:
#cloning all items that were not present on the portal before
for item in items_to_be_cloned:    
    try:
        with tempfile.TemporaryDirectory() as temp_dir:
            thumbnail_file = item.download_thumbnail(temp_dir)
            metadata_file = item.download_metadata(temp_dir)
            target_item_properties = {'title': item.title,
                                      'tags': item.tags,
                                      'text':item.get_data(True),
                                      'type':item.type,
                                      'url':item.url
                                     }       
            #create an item
            target_item = target.content.add(target_item_properties, thumbnail=thumbnail_file)
            
            #share that item with the group on the target portal
            target_item.share(groups=[target_group])
            
            #display the item
            display(target_item)
            
    except Exception as e:
        print('Item {} could not be created in the target portal'.format(item.title))
        print(e)